# Milky Way Mapper's Galaxy

## Section 7: Paper Task

In [1]:
#Import some things
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import astropy.coordinates as coord
import astropy.units as u
from astropy.io import fits, ascii
from astropy.table import Table
from astropy.coordinates import SkyCoord
%matplotlib inline


In [2]:
#load in the data (may have to change this for wherever you downloaded your file)
#in google colab you can get the file using
#!wget https://dr19.sdss.org/sas/dr19/spectro/astra/0.6.0/summary/astraAllStarASPCAP-0.6.0.fits.gz 

filename='astraAllStarASPCAP-0.6.0.fits'
tb = fits.open(filename)
header=tb[2].header
data = tb[2].data 

In [3]:
good=np.where((data['teff'] > 3700) & (data['teff'] < 5300) &
               (data['logg'] > 0.9) & (data['logg'] < 3.3) &
               (data['m_h_atm'] > -2.0) & (data['m_h_atm'] < 0.6) &   
               (data['flag_bad']==False) )

data_masked=data[good]

## Training Sets

In [4]:
# #TESS
# tessraw = Table.read("Theodoridis2025.csv", format="ascii")
# #this one has an age column in Gyr already so we're just going to rename it Age
# tessraw['Final_age'].name='Age'
# hasagetess=np.where((tessraw['Age']==tessraw['Age']) & (tessraw['Age']>0.1) &(tessraw['Flag']==0))
# tess=tessraw[hasagetess]
# tess

In [5]:
# Age option 2 APOKASC-2 Pinsonneault et al. 2018
#Reading in the table, making sure all the tables have a column named Age in Gyr
# and that every star in the table has an Age
apokasc2raw = Table.read("Pinsonneault2018.txt", format="ascii.cds")
apokasc2raw['Age']=(10**np.array(apokasc2raw['LogAge'])/1000.) #Age was in log(Myr) so needs converting
hasagea2=np.where((apokasc2raw['Age']==apokasc2raw['Age']) & (apokasc2raw['Age']>0.1))
apokasc2=apokasc2raw[hasagea2]
apokasc2

KIC,2MASS,Teff,e_Teff,FeH,e_FeH,AFe,e_AFe,Nmax,e_Nmax,Dnu,e_Dnu,ES,Fdnu,e_Fdnu,M(cor),e_M(cor)-ran,e_M(cor)-sys,R(cor),e_R(cor)-ran,e_R(cor)-sys,logg(seis),e_logg(seis)-ran,e_logg(seis)-sys,Rho,e_Rho-ran,e_Rho-sys,LogAge,E_LogAge,e_LogAge,Av,e_Av,Notes,Age
,,K,K,dex(---),dex(---),dex(---),dex(---),uHz,,uHz,,,,,Msun,,,Rsun,,,dex(cm / s2),,,g / cm3,,,Myr,Myr,Myr,mag,mag,,
int64,str18,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str18,float64
1027110,2M19250937+3644599,4177.6,51.8,-0.232,0.025,0.205,0.015,6.496,0.015,1.132,0.029,RGB,1.0458,0.0006,0.985,0.126,0.044,23.412,0.06,0.017,1.692,0.007,0.007,0.0001082,0.058,0.008,4.002,0.205,-0.189,0.269,0.121,SeisUnc,10.046157902783946
1027337,2M19252021+3647118,4636.0,67.3,0.275,0.024,0.023,0.01,73.975,0.009,6.991,0.013,RGB,1.0333,0.0003,1.227,0.063,0.038,7.544,0.029,0.014,2.772,0.005,0.007,0.004029,0.026,0.007,3.798,0.099,-0.09,0.214,0.071,...,6.2805835881331795
1160789,2M19233280+3652207,4729.6,72.3,-0.257,0.034,0.188,0.015,25.209,0.015,3.545,0.011,RC,0.9965,0.0324,0.875,0.147,0.082,10.86,0.071,0.025,2.308,0.007,0.01,0.0009635,0.069,0.008,3.889,0.133,-0.146,0.009,0.074,...,7.744617978025183
1161447,2M19241746+3651460,4776.1,86.2,0.058,0.029,-0.006,0.013,37.066,0.027,4.153,0.011,RC,1.003,0.0238,1.46,0.135,0.076,11.54,0.059,0.022,2.478,0.012,0.01,0.00134,0.052,0.008,3.396,0.166,-0.145,0.4,0.086,...,2.48885731828239
1161618,2M19242614+3648478,4742.0,72.1,0.064,0.029,0.005,0.012,33.926,0.01,4.093,0.012,RC,1.001,0.0033,1.183,0.063,0.077,10.879,0.028,0.023,2.438,0.005,0.01,0.001296,0.026,0.008,3.639,0.064,-0.069,0.199,0.074,...,4.355118736855684
1162220,2M19245791+3653298,4190.1,51.7,0.083,0.021,0.07,0.011,11.0,0.01,1.669,0.011,RGB,1.0484,0.0004,1.007,0.055,0.044,18.175,0.024,0.017,1.922,0.005,0.007,0.0002364,0.021,0.008,4.056,0.087,-0.083,0.181,0.073,...,11.37627285823431
1162746,2M19252639+3649116,4798.1,75.6,-0.388,0.038,0.229,0.017,27.798,0.015,3.763,0.01,RC,0.9972,0.0281,0.941,0.131,0.08,10.688,0.062,0.024,2.354,0.007,0.01,0.001087,0.06,0.008,3.786,0.139,-0.131,0.172,0.075,...,6.109420249055721
1163114,2M19254564+3650475,4285.8,54.4,0.297,0.02,0.025,0.01,14.356,0.011,1.887,0.008,RGB,1.0396,0.0004,1.467,0.05,0.043,19.097,0.021,0.016,2.042,0.006,0.007,0.0002971,0.016,0.008,3.538,0.079,-0.077,0.382,0.069,...,3.451437393358561


In [6]:
# # Age option 3 APOKASC-3 Pinsonneault et al. 2025
# #Reading in the table, making sure all the tables have a column named Age in Gyr
# # and that every star in the table has an Age
# apokasc3raw= Table.read("Pinsonneault2025.txt", format="ascii.cds")
# #in this case there were two age columns, one for Red Clump and one for Red Giant Branch so we combine them
# ageRC=np.array(apokasc3raw['AgeRC']*(apokasc3raw['EvolState']=='RC'))
# rcnans=np.isnan(ageRC) #removing nans from this version of the table.
# ageRC[rcnans]=0
# ageRGB=np.array(apokasc3raw['AgeRGB']*(apokasc3raw['EvolState']=='RGB'))
# rgbnans=np.isnan(ageRGB) #removing nans from this version of the table.
# ageRGB[rgbnans]=0
# apokasc3raw['Age']=(ageRC+ageRGB)

# hasagea3=np.where((apokasc3raw['Age']==apokasc3raw['Age']) & (apokasc3raw['Age']>0.1))
# apokasc3=apokasc3raw[hasagea3]
# apokasc3

In [7]:
# Age option 4 K2 data Warfield et al. 2024
#Reading in the table, making sure all the tables have a column named Age in Gyr
# and that every star in the table has an Age
apok2raw = Table.read("Warfield2024.txt", format="ascii.cds")
#this one has an age column in Gyr already so we're just going to rename it Age
hasageapok2=np.where((apok2raw['Age']==apok2raw['Age']) & (apok2raw['Age']>0.1))
apok2=apok2raw[hasageapok2]
apok2

EPIC,APOGEE,GaiaEDR3,RAdeg,DEdeg,GLON,GLAT,Rkpc,Zkpc,Teff,Teff-u,e_Teff,logg,logg-u,e_logg,[Fe/H],e_[Fe/H],[a/M],e_[a/M],[O/Fe],e_[O/Fe],aFlag,Mstar,e_Mstar,Rstar,e_Rstar,logg-seis,e_logg-seis,numax,e_numax,deltanu,e_deltanu,Age,e_Age,E_Age,Age-mode,SFW,EvState
,,,deg,deg,deg,deg,kpc,kpc,K,K,K,dex(cm / s2),dex(cm / s2),dex(cm / s2),dex(Sun),dex(Sun),dex(Sun),dex(Sun),dex(Sun),dex(Sun),,Msun,Msun,Rsun,Rsun,dex(cm / s2),dex(cm / s2),uHz,uHz,uHz,uHz,Gyr,Gyr,Gyr,Gyr,,
int64,str18,int64,float64,float64,float64,float64,str6,str6,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,str6,str5,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,str3
211821580,2M08270102+1737253,662088729307609216,126.7543,17.6237,206.812,28.792,10.252,1.306,4795,4696,9,2.43,2.5,0.03,-0.453,0.008,0.094,0.007,0.173,0.02,0,0.92,0.08,9.5,0.26,2.45,0.01,34.4,0.9,4.42,0.01,11.3,3.0,4.1,10.8,4.32,RGB
212434736,2M13572534-1408010,6302108283398593152,209.3556,-14.1336,326.169,45.74,7.504,0.795,4787,4705,9,2.94,2.99,0.02,-0.24,0.007,0.183,0.006,0.252,0.018,1,1.11,0.02,5.66,0.04,2.98,0.0,116.8,0.7,10.56,0.0,7.3,0.5,0.5,7.2,7.43,RGB
212333123,2M13400342-1634554,3605202458865639552,205.0143,-16.5821,319.433,44.726,6.713,2.016,4795,4703,16,2.87,2.95,0.04,-0.383,0.014,0.235,0.011,0.312,0.036,1,0.87,0.06,5.79,0.13,2.85,0.0,87.3,0.7,9.03,0.1,16.8,4.0,5.3,17.5,5.61,RGB
212173678,2M08253330+2334276,678233202137154304,126.3888,23.5743,200.25,30.532,9.024,0.587,4780,4701,8,2.76,2.83,0.02,-0.188,0.007,0.063,0.006,0.103,0.017,0,1.27,0.05,7.56,0.11,2.78,0.0,74.9,0.9,7.31,0.03,4.3,0.6,0.7,4.1,6.53,RGB
213789445,2M19182284-2816172,6759783812391009920,289.5952,-28.2715,9.756,-17.933,6.271,-0.595,4438,4329,7,2.1,2.21,0.02,-0.239,0.009,0.087,0.006,0.124,0.013,0,1.38,0.39,18.24,1.68,2.06,0.03,14.6,1.0,2.04,0.06,3.0,2.1,6.8,2.7,3.24,RGB
212591385,2M13374700-1046588,3616757776317838464,204.4458,-10.783,320.985,50.457,6.843,2.143,4507,4404,13,2.32,2.43,0.03,-0.173,0.014,0.178,0.009,0.226,0.024,1,0.88,0.07,10.42,0.26,2.35,0.01,28.4,0.4,3.78,0.04,18.3,4.6,6.2,17.5,11.17,RGB
210521826,2M04034090+1546195,45403134375970432,60.9204,15.7721,176.149,-26.646,9.781,-0.808,4634,4519,9,2.28,2.36,0.03,-0.361,0.009,0.097,0.007,0.114,0.018,0,1.05,0.14,12.44,0.52,2.27,0.01,23.3,0.5,3.16,0.06,7.5,2.9,4.7,6.4,5.51,RGB
246067499,2M23464240-0751238,2436395519498043520,356.6767,-7.8566,81.44,-65.372,8.085,-0.616,4678,4613,10,2.95,3.06,0.02,0.171,0.009,0.018,0.006,0.034,0.018,0,1.11,0.08,5.3,0.13,3.03,0.0,135.1,0.7,11.67,0.14,9.3,2.3,3.0,9.2,41.76,RGB


In [8]:
agedata= apokasc2
agedata2= apok2

In [9]:
#Option 1 TESS Theodoridis et al. 2025
# intersect2, ind_a2, ind_b2 = np.intersect1d(data_masked['tic_v8_id'],agedata2['TIC'], return_indices=True)

#Option 2 APOKASC-2 Pinsonneault et al. 2018
intersect, ind_a, ind_b = np.intersect1d(data_masked['sdss4_apogee_id'],agedata['2MASS'], return_indices=True) 

#Option 3 APOKASC-3 Pinsonneault et al. 2025
#intersect, ind_a, ind_b = np.intersect1d(data_masked['gaia_dr3_source_id'],agedata['GaiaDR3'], return_indices=True) 

#Option 4 APO-K2 Warfield et al. 2024
intersect2, ind_a2, ind_b2 = np.intersect1d(data_masked['sdss4_apogee_id'],agedata2['APOGEE'], return_indices=True) 

print(len(ind_b), len(ind_b2))

5993 4451


In [10]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [11]:
from tensorflow import keras

## First Training Set

In [12]:
fullx = np.dstack([data_masked['teff'][ind_a],data_masked['logg'][ind_a], data_masked['m_h_atm'][ind_a],
                   data_masked['alpha_m_atm'][ind_a], data_masked['c_h'][ind_a], data_masked['n_h'][ind_a]])[0]

fully = np.dstack([agedata['Age'][ind_b]])[0] #for Pinsonneault 2018

#remove non-finite entries!
mask = np.all(np.isfinite(fullx), axis=1) & np.all(np.isfinite(fully), axis=1)
fullx, fully = fullx[mask], fully[mask]

scaling_x = np.median(fullx, axis=0)
scaling_y = np.median(fully, axis=0)

fullx, fully = fullx/scaling_x, fully/scaling_y

In [13]:
neurons_per_layer=20
layers=5
iterations=200

In [14]:

#start with an input layer
inputs = keras.Input(shape=(6,))
#now we add the Dense layers (indicating the previous layer in the brackets following the layer declaration

#change this part if you're changing the number of layers
layer1 =keras.layers.Dense(neurons_per_layer, activation='relu')(inputs)
layer2 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer1)
layer3 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer2)
layer4 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer3)
layer5 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer4)

#then the output layer YOU ALSO HAVE TO MAKE THIS MATCH YOUR NUMBER OF LAYERS
outputs = keras.layers.Dense(1)(layer5)


# then we put that all together in the Model object
model = keras.Model(inputs=inputs, outputs=outputs, name='test')
#and we can print a summary to check it all went to plan
model.summary()

Model: "test"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 20)             │           140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,841 (7.19 KB)

 Trainable params: 1,841 (7.19 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [16]:
tenpercent=len(agedata['Age'][ind_b])//10 #figure out what ten percent of this set of age data is

#last name before M 
#trainbin=slice(0,-1*tenpercent-1)
#testing=slice(-1*tenpercent,-1)


#last name M or later
trainbin=slice(tenpercent+1,-1)
testing=slice(0,tenpercent)


x_train, y_train = fullx[trainbin], fully[trainbin]
x_test, y_test = fullx[testing], fully[testing]

In [17]:
model.fit(x_train, y_train, epochs=iterations, validation_split=0.05, batch_size=300)

Epoch 1/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.0000e+00 - loss: 1.5923 - val_accuracy: 0.0000e+00 - val_loss: 0.7158
Epoch 2/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 2.0354e-04 - loss: 0.7432 - val_accuracy: 0.0039 - val_loss: 0.5393
Epoch 3/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 8.1417e-04 - loss: 0.5893 - val_accuracy: 0.0039 - val_loss: 0.5189
Epoch 4/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0012 - loss: 0.5137 - val_accuracy: 0.0039 - val_loss: 0.4270
Epoch 5/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0012 - loss: 0.4536 - val_accuracy: 0.0039 - val_loss: 0.3761
Epoch 6/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0012 - loss: 0.4065 - val_accuracy: 0.0039 - val_loss: 0.3381
Epoch 7/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0012 - loss: 0.3808 - val_accuracy: 0.0039 - val_loss: 0.3140
Epoch 8/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0012 - loss: 0.3631 - val_acc

In [18]:
predictions = model.predict(x_test)
print(len(predictions))

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
599


In [19]:
metric=0.3 #is the accuracy better than 30%?
goodfit=np.where(((1-metric) < predictions/y_test) & ((1+metric) > predictions/y_test)) 
badfit=np.where(((1-metric) > predictions/y_test) | ((1+metric) < predictions/y_test))

print ('With ', neurons_per_layer, 'neurons per layer, ', layers, 'layers, and ', iterations, 'iterations')
print ('using the training set', trainbin)
print (len(goodfit[0])/len(y_test)*100, 'percent of the ages are good')
print (len(badfit[0])/len(y_test)*100, 'percent of the ages are bad')

With  20 neurons per layer,  5 layers, and  200 iterations
using the training set slice(600, -1, None)
65.6093489148581 percent of the ages are good
34.390651085141904 percent of the ages are bad


In [20]:
DR19x = np.dstack([data_masked['teff'],data_masked['logg'], data_masked['m_h_atm'],
                   data_masked['alpha_m_atm'], data_masked['c_h'], data_masked['n_h']])[0]
print(len(data_masked['teff']))

DR19x= DR19x/scaling_x
predictionsDR19 = model.predict(DR19x)

425631
13301/13301 ━━━━━━━━━━━━━━━━━━━━ 18s 1ms/step


## Training Set 2

In [21]:
fullx2 = np.dstack([data_masked['teff'][ind_a2],data_masked['logg'][ind_a2], data_masked['m_h_atm'][ind_a2],
                   data_masked['alpha_m_atm'][ind_a2], data_masked['c_h'][ind_a2], data_masked['n_h'][ind_a2]])[0]

fully2 = np.dstack([agedata2['Age'][ind_b2]])[0] #for Pinsonneault 2018

#remove non-finite entries!
mask2 = np.all(np.isfinite(fullx2), axis=1) & np.all(np.isfinite(fully2), axis=1)
fullx2, fully2 = fullx2[mask2], fully2[mask2]

scaling_x2 = np.median(fullx2, axis=0)
scaling_y2 = np.median(fully2, axis=0)
fullx2, fully2 = fullx2/scaling_x2, fully2/scaling_y2

In [22]:
tenpercent2=len(agedata2['Age'][ind_b2])//10 #figure out what ten percent of this set of age data is

#last name before M 
#trainbin=slice(0,-1*tenpercent-1)
#testing=slice(-1*tenpercent,-1)


#last name M or later
trainbin2=slice(tenpercent2+1,-1)
testing2=slice(0,tenpercent2)


x_train2, y_train2 = fullx2[trainbin2], fully2[trainbin2]
x_test2, y_test2 = fullx2[testing2], fully2[testing2]

In [23]:
model.fit(x_train2, y_train2, epochs=iterations, validation_split=0.05, batch_size=300)

Epoch 1/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0086 - loss: 0.9004 - val_accuracy: 0.0102 - val_loss: 0.6126
Epoch 2/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0094 - loss: 0.6228 - val_accuracy: 0.0102 - val_loss: 0.5414
Epoch 3/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0094 - loss: 0.5864 - val_accuracy: 0.0102 - val_loss: 0.5437
Epoch 4/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0094 - loss: 0.5804 - val_accuracy: 0.0102 - val_loss: 0.5389
Epoch 5/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0094 - loss: 0.5775 - val_accuracy: 0.0102 - val_loss: 0.5375
Epoch 6/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0094 - loss: 0.5718 - val_accuracy: 0.0102 - val_loss: 0.5373
Epoch 7/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0094 - loss: 0.5685 - val_accuracy: 0.0102 - val_loss: 0.5358
Epoch 8/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0094 - loss: 0.5646 - val_accuracy: 0.

In [24]:
predictions2 = model.predict(x_test2)
print(len(predictions2))

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
445


In [25]:
metric2=0.3 #is the accuracy better than 30%?
goodfit2=np.where(((1-metric2) < predictions2/y_test2) & ((1+metric2) > predictions2/y_test2)) 
badfit2=np.where(((1-metric2) > predictions2/y_test2) | ((1+metric2) < predictions2/y_test2))

print ('With ', neurons_per_layer, 'neurons per layer, ', layers, 'layers, and ', iterations, 'iterations')
print ('using the training set', trainbin)
print (len(goodfit2[0])/len(y_test2)*100, 'percent of the ages are good')
print (len(badfit2[0])/len(y_test2)*100, 'percent of the ages are bad')

With  20 neurons per layer,  5 layers, and  200 iterations
using the training set slice(600, -1, None)
47.640449438202246 percent of the ages are good
52.359550561797754 percent of the ages are bad


In [26]:
DR19x2 = np.dstack([data_masked['teff'],data_masked['logg'], data_masked['m_h_atm'],
                   data_masked['alpha_m_atm'], data_masked['c_h'], data_masked['n_h']])[0]
print(len(data_masked['teff']))

DR19x2= DR19x2/scaling_x2
predictionsDR19_2 = model.predict(DR19x2)

425631
13301/13301 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step


In [27]:
import pandas as pd
#Pulling Labels and Data
apokasc2_age = predictionsDR19.flatten() * scaling_y
apok2_age = predictionsDR19_2.flatten() * scaling_y2
age_difference = apok2_age - apokasc2_age
star_ids = data_masked['tic_v8_id']
#Creating DataFrame of the results
dataframe = pd.DataFrame({
	"TIC": star_ids,
	'APOK2_predicted_Gyr': apok2_age,
	'APOKASC2_predicted_Gyr': apokasc2_age,
    'APOK2_APOKASC2_Diff': age_difference
})
#Saving DataFrame to CSV
dataframe.to_csv('APOK2_APOKASC2_Comparison.csv', index=False)

In [28]:
print(f'Average difference between data set is:', np.mean(age_difference))

Average difference between data set is: 5.5876219185966
